In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
cols = ['ID(original)', 'ID' , 'Q' , 'A']
data = pd.read_csv('COQB-19_crowdsourced_06202020.csv', sep=',', names=cols, header=None, encoding='latin-1',skiprows=1)
# remove nans
data = data.dropna()
Questions = data['Q']
ID= data['ID'].tolist()
corpus = Questions.tolist()
Answers =data['A'].tolist()

In [ ]:
lemmatizer = WordNetLemmatizer()
def stemSentence(sentence):
    token_words= word_tokenize(sentence)
    lematized_tokens = [lemmatizer.lemmatize(w) for w in token_words]
    return "  ".join(lematized_tokens)

In [ ]:
stemmed_corpus = []
SC = []
for i in range(len(corpus)):
  stemmed_corpus.append((i,stemSentence(corpus[i])))
  SC.append(stemSentence(corpus[i].lower()))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [ ]:
X = vectorizer.fit_transform(SC)

In [ ]:
mat_x = X.toarray()

In [ ]:
def answer_question(question):

  """  returns (top_idx,  top_idx_score, all_scores)
  top_idx: index of most similar question
  top_idx_score:  score of most similar question.
  all_scores: scores of all candidates.
  """
  lematized_question = stemSentence(question)
  question_x = vectorizer.transform([lematized_question])
  sim_scores = cosine_similarity(question_x,  mat_x).ravel()
  top_idx = np.argmax(sim_scores)
  return Answers[stemmed_corpus[top_idx][0]]

In [ ]:
import os

if not os.path.exists('templates'):
  os.makedirs('templates')
if not os.path.exists('static'):
  os.makedirs('static')

In [ ]:
home= '''

<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <title>InferMedica Chatbot</title>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <meta http-equiv="X-UA-Compatible" content="ie=edge">
  <link rel="stylesheet" href="/static/style.css">
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.2.1/jquery.min.js"></script>
</head>

<body>
  <!-- partial:index.partial.html -->
  <section class="msger">
    <header class="msger-header">
      <div class="msger-header-title">
        <i></i> InferMedica Chatbot <i></i>
      </div>
    </header>

    <main class="msger-chat">
      <div class="msg left-msg">
        <div class="msg-img" style="background-image: url(https://image.flaticon.com/icons/svg/327/327779.svg)"></div>

        <div class="msg-bubble">
          <div class="msg-info">
            <div class="msg-info-name">InferMedica</div>
        
          </div>

          <div class="msg-text">
            Hi, welcome to InferMedica ChatBot! How can I help you?! 😄
          </div>
        </div>
      </div>

    </main>

    <form class="msger-inputarea">
      <input type="text" class="msger-input" id="textInput" placeholder="Type your message...">
      <button type="submit" class="msger-send-btn">Send</button>
    </form>
  </section>
  <!-- partial -->
  <script src='https://use.fontawesome.com/releases/v5.0.13/js/all.js'></script>
  <script>

    const msgerForm = get(".msger-inputarea");
    const msgerInput = get(".msger-input");
    const msgerChat = get(".msger-chat");


    // Icons made by Freepik from www.flaticon.com
    const BOT_IMG = "https://image.flaticon.com/icons/svg/327/327779.svg";
    const PERSON_IMG = "https://image.flaticon.com/icons/svg/145/145867.svg";
    const BOT_NAME = "ChatBot";
    const PERSON_NAME = "You";

    msgerForm.addEventListener("submit", event => {
      event.preventDefault();

      const msgText = msgerInput.value;
      if (!msgText) return;

      appendMessage(PERSON_NAME, PERSON_IMG, "right", msgText);
      msgerInput.value = "";
      botResponse(msgText);
    });

    function appendMessage(name, img, side, text) {
      //   Simple solution for small apps
      const msgHTML = `
<div class="msg ${side}-msg">
  <div class="msg-img" style="background-image: url(${img})"></div>

  <div class="msg-bubble">
    <div class="msg-info">
      <div class="msg-info-name">${name}</div>
      <div class="msg-info-time">${formatDate(new Date())}</div>
    </div>

    <div class="msg-text">${text}</div>
  </div>
</div>
`;

      msgerChat.insertAdjacentHTML("beforeend", msgHTML);
      msgerChat.scrollTop += 500;
    }

    function botResponse(rawText) {

      // Bot Response
      $.get("/get", { msg: rawText }).done(function (data) {
        console.log(rawText);
        console.log(data);
        const msgText = data;
        appendMessage(BOT_NAME, BOT_IMG, "left", msgText);

      });

    }


    // Utils
    function get(selector, root = document) {
      return root.querySelector(selector);
    }

    function formatDate(date) {
      const h = "0" + date.getHours();
      const m = "0" + date.getMinutes();

      return `${h.slice(-2)}:${m.slice(-2)}`;
    }



  </script>

</body>

</html>
'''


file=open("templates/home.html", "w")
file.write(home)
file.close()


In [ ]:
!pip install flask-ngrok
from flask import Flask, render_template, request, redirect, url_for
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)   
  
app.static_folder = 'static'
@app.route("/")
def home():
    return render_template("home.html")
@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return answer_question(userText)
    
app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2014cf7031af.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Jul/2021 16:50:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 16:50:38] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 16:50:38] "GET /static/medical.jpg HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 16:50:43] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Jul/2021 16:50:43] "GET /get?msg=hi HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 16:50:48] "GET /get?msg=what%20is%20your%20name%3F HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 16:50:56] "GET /get?msg=how%20are%20you%3F HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 16:51:02] "GET /get?msg=who%20are%20you%3F HTTP/1.1" 200 -
